In [ ]:
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. 경로 설정
FF_BASE_PATH = Path("/content/drive/MyDrive/HECTO/Dataset/FF_frames")
CSV_SAVE_PATH = "/content/drive/MyDrive/HECTO/Dataset/CSV/ff_metadata.csv"
data_list = []

print("🔍 FF++ 폴더 구조 분석 중...")

🔍 FF++ 폴더 구조 분석 중...


In [ ]:
# 2. 이미지 파일 탐색 (rglob 사용)
image_files = list(FF_BASE_PATH.rglob('*.jpg'))

for img_path in image_files:
    # 상대 경로 추출 (예: Real/001/f000_0.jpg 또는 Fake/Deepfakes/001_002/f000_1.jpg)
    rel_path = img_path.relative_to(FF_BASE_PATH)
    parts = rel_path.parts

    # 레이블 결정
    is_real = 'real' in [p.lower() for p in parts]
    label = 0 if is_real else 1

    # Video ID 추출: 영상별로 고유해야 하므로 폴더 경로를 조합
    # 예: Real_001, Fake_Deepfakes_001_002
    video_id = "FF_" + "_".join(parts[:-1])

    if is_real:
        method = 'Original'
    else:
        # Fake인 경우 두 번째 폴더명(Deepfakes, FaceSwap 등)을 가져옴
        method = parts[1]

    # 프레임 인덱스 추출 (f005_0.jpg -> 5)
    try:
        # 파일명이 f000_0.jpg 형태이므로 'f'를 떼고 '_'로 분리
        frame_idx = int(img_path.stem.split('_')[0].replace('f', ''))
    except:
        frame_idx = 0

    data_list.append({
        'dataset_name': 'FF++',
        'video_id': video_id,
        'frame_idx': frame_idx,
        'frame_path': str(img_path),
        'label': label,
        'bbox': [0, 0, 256, 256],
        'detection_score': 1.0,
        'original_video_width': 1920, # FF++ 기본 해상도
        'original_video_height': 1080,
        'manipulation_method': parts[1] if label == 1 else 'None' # FF++는 조작방식 추가 기록 가능
    })

df = pd.DataFrame(data_list)

In [ ]:
# 3. 비디오 단위 Split (8:1:1)
print(f"✂️ 총 {df['video_id'].nunique()}개의 FF++ 영상을 분할 중...")

unique_vids = df['video_id'].unique()
train_vids, temp_vids = train_test_split(unique_vids, test_size=0.2, random_state=42)
val_vids, test_vids = train_test_split(temp_vids, test_size=0.5, random_state=42)

df.loc[df['video_id'].isin(train_vids), 'split'] = 'train'
df.loc[df['video_id'].isin(val_vids), 'split'] = 'val'
df.loc[df['video_id'].isin(test_vids), 'split'] = 'test'

✂️ 총 1984개의 FF++ 영상을 분할 중...


In [ ]:
# 4. 저장
df.to_csv(CSV_SAVE_PATH, index=False)
print(f"✅ FF++ CSV 생성 완료: {CSV_SAVE_PATH}")

# 통계 출력
print("\n--- [FF++ 데이터셋 통계] ---")
print(df.groupby(['split', 'label']).size().unstack(fill_value=0))

✅ FF++ CSV 생성 완료: /content/drive/MyDrive/HECTO/Dataset/CSV/ff_metadata.csv

--- [FF++ 데이터셋 통계] ---
label      0      1
split              
test    1395   1554
train  11806  11423
val     1394   1508
